
# MedGemma: Prototype AI clinical scribe:
Modified from Notebook: https://github.com/Google-Health/medgemma/blob/main/notebooks/quick_start_with_hugging_face.ipynb


Tutorial: Using Medgemma as a prototype AI Clinical Scribe.AI clinical scribes are emerging technologies that leverage natural language processing and machine learning to automatically listen to and transcribe patient-clinician conversations, converting them into structured medical notes for electronic health records (EHRs). Their primary aim is to alleviate the significant administrative burden on healthcare providers, freeing them from extensive data entry to focus more on direct patient care and interaction. By automating documentation, these AI tools promise to reduce clinician burnout, improve the efficiency of clinical workflows, and potentially enhance the accuracy and completeness of medical records, ultimately aiming to improve the overall quality of patient encounters.

Learn more about the model at the [HAI-DEF developer site](https://developers.google.com/health-ai-developer-foundations/medgemma).

###Note: Make sure you have an HF_Token from huggingface so you can access all the models. Put the HF_Token into google secrets on the left - see the key.

In [1]:
import os
import sys
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
from IPython.display import display, Markdown

### Install dependencies

In [2]:
! pip install --upgrade --quiet accelerate bitsandbytes transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 25.0 MB/s eta 0:00:00


###Note Go to the model page on hugging face for google/medgemma-4b-it to get permission to use the model.

In [3]:
from transformers import BitsAndBytesConfig
import torch

model_id = f"google/medgemma-4b-it"

model_kwargs = dict(
  torch_dtype=torch.bfloat16,
  device_map="auto",
)
model_kwargs["quantization_config"] = BitsAndBytesConfig(load_in_4bit=True)

###Run model with the `pipeline` API

In [4]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model_id,
    model_kwargs=model_kwargs,
)

pipe.model.generation_config.do_sample = False

config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Device set to use cuda:0


###Hypothetical clinic audio transcript - this was generated by gemini 2.5 pro and lightly edited

In [5]:
ts="""
Dr. Carter: Hi Mr. Miller, I'm Dr. Carter. What brings you in today?
Patient Hi Doctor. It's my throat. It's… it's been killing me for the last few days, but today it's just unbearable.
Dr. Carter: Okay, tell me more about it. When did it start?
Patient: Uh, maybe four days ago? It started just like a regular sore throat, scratchy. I figured it was a cold. But it's gotten progressively worse, especially the last 24-48 hours. The pain is… intense.
Dr. Carter: On a scale of 1 to 10, 10 being the worst pain imaginable, what would you rate it right now?
Patient: Honestly, like a 9. Especially when I try to swallow.
Dr. Carter: And is the pain localized to one side, or all over?
Patient: It feels much worse on the left side. Like, deep in there.
Dr. Carter: Any trouble swallowing? Food? Liquids? Even your own saliva?
Patient: Yeah, all of it. Swallowing saliva is agony. I’ve barely eaten anything today. I’m kind of… drooling a bit because it hurts so much to swallow. It’s embarrassing.
Dr. Carter: Not embarrassing at all, Mr. Miller. It just tells me how much discomfort you're in. Have you noticed any change in your voice?
Patient: My wife said I sound funny.
Dr. Carter: Any fever, chills, body aches?
Patient: Yeah, I felt really hot last night, definitely had a fever. Took some ibuprofen. Maybe some mild body aches, but the throat is the main thing.
Dr. Carter: Have you tried any other remedies? Lozenges, sprays?
Patient: Lozenges don't touch it. Salt water gargle felt like fire.
Dr. Carter: Any difficulty opening your mouth wide?
Patient: Yeah, it’s a bit stiff. Doesn’t want to open all the way.
Dr. Carter: Okay. Any history of recurrent tonsillitis or similar severe sore throats?
Patient: I used to get strep a lot as a kid, but nothing like this as an adult.
Dr. Carter: Any allergies to medications?
Patient: Penicillin. I get a rash.
Dr. Carter: Good to know. Any other medical conditions? Are you a smoker?
Patient: No other conditions. I vape, but I haven't been able to the last couple of days.
Dr. Carter: Okay, Mr. Miller. I'm going to take a look. I'll also check your vital signs.Temp 38.9°C, HR 110, BP 130/80, RR 18, SpO2 98% on room air
Dr. Carter: Your temperature is up, and your heart rate is a bit fast, which is common with pain and infection. Okay, let’s have a look in your throat. Can you open as wide as you can for me? Say "Ahhh."
Patient:Arrr…
Dr. Carter: Okay, just relax your tongue. Hmm. Yes, I see. There’s significant swelling on the left side, around your tonsil. The tonsil itself is very red and pushed towards the midline. Your uvula, that little punching bag thing in the back, is deviated over to the right. That whole left peritonsillar area looks very angry and full.
Patient: Is it bad?
Dr. Carter: It looks like you have what we call a peritonsillar abscess, Mr. Miller. That means there's a collection of pus that has formed in the tissues next to your tonsil. That explains the severe one-sided pain, the difficulty swallowing, the change in your voice, and the trouble opening your mouth – that’s called trismus.
Patient: An abscess? Like… pus? Oh man. What does that mean? Do I need surgery?
Dr. Carter: It means we need to get that pus out. The standard treatment is to drain it. Sometimes this can be done with a needle aspiration or a small incision. Given the extent of it and your symptoms, this is something that typically needs to be handled by an Ear, Nose, and Throat specialist (ENT).
Patient: So, what happens now?
Dr. Carter: Here’s the plan:
We need to get you some immediate relief and start treating the infection. I’m going to give you an IV. Through that, we'll give you a strong antibiotic – since you're allergic to penicillin, we'll use something like Clindamycin.
We’ll also give you some IV steroids to help with the swelling, and some IV pain medication to make you more comfortable.
I'm going to call the on-call ENT specialist at University Hospital right now. They will likely want to see you in the Emergency Department there to evaluate you for drainage. This isn't something we typically drain here in urgent care when it's this significant.
Patient: University Hospital? So I have to go to the ED?
Dr. Carter: Yes, that's the safest and best place for this to be managed. They have the specialized equipment and expertise. We'll get you started on treatment here to make the transfer more comfortable and to get a head start on the infection. The main thing is to get that abscess drained to provide relief and prevent complications.
Patient:Okay. Wow. I didn't think a sore throat could get this bad.
Dr. Carter: It's not a simple sore throat anymore, unfortunately. The good news is that once it's drained, patients usually feel a significant improvement in pain very quickly. Do you have any questions for me right now?
Patient: Um… will it hurt when they drain it?
Dr. Carter: They will numb the area very well before they do anything. You'll feel pressure, but it shouldn't be sharply painful. And the relief afterwards is usually substantial. We'll get your IV started, get those medications in, and I'll get on the phone with ENT. We'll get you sorted out.
Patient: Okay. Thank you, Doctor.
Dr. Carter: You're welcome, Mr. Miller.
"""

In [6]:
ma="""
You are an expert medical assistant. Your task is to convert the following clinical transcription into a structured SOAP note.
    The SOAP note should have four distinct sections: Subjective, Objective, Assessment, and Plan.
    Ensure that information from the transcription is accurately placed into the correct section and easy to read paragraphs. Do not add
    any information that is not explicitly mentioned in the transcription. Make subjective section easy to read like a paragraph.
"""

In [7]:
system_instruction= ma
prompt = ts
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": system_instruction}]
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": prompt},

        ]
    }
]



#Find the Answer using your chosen model

In [8]:
output = pipe(text_inputs=messages, max_new_tokens=1000)
response = output[0]["generated_text"][-1]["content"]

#display(Markdown(f"\n\n---\n\n**[ User ]**\n\n{prompt}"))
display(Markdown(f"\n\n---\n\n**[ MedGemma ]**\n\n{response}\n\n---\n\n"))



---

**[ MedGemma ]**

Here's the SOAP note based on the provided transcription:

**Subjective:**

Patient presents with a severe, 4-day history of sore throat, initially scratchy, progressing to intense pain, especially on the left side, with difficulty swallowing (saliva, food, and own saliva), drooling, and a muffled voice. Patient reports a fever, mild body aches, and a temperature of 38.9°C. Patient reports the pain to be a 9/10. Patient has a history of recurrent strep infections as a child, but not similar to this current presentation. Patient is allergic to penicillin. Patient reports no other medical conditions. Patient has been vaping in the last couple of days. Patient denies any other symptoms.

**Objective:**

*   Temperature: 38.9°C (101.8°F)
*   Heart Rate: 110 bpm
*   Blood Pressure: 130/80 mmHg
*   Respiratory Rate: 18 breaths/min
*   Oxygen Saturation: 98% on room air
*   Physical Exam: Significant swelling on the left side of the throat, around the tonsil. Tonsil itself is red and pushed towards the midline. Uvula deviated to the right. Trismus (difficulty opening mouth wide).

**Assessment:**

Peritonsillar abscess (left side).

**Plan:**

1.  Initiate intravenous (IV) fluids with antibiotics (Clindamycin, penicillin allergy).
2.  Administer IV steroids to reduce swelling.
3.  Administer IV pain medication.
4.  Contact University Hospital ENT on-call service to facilitate transfer to the Emergency Department for drainage of the peritonsillar abscess.
5.  Patient to be transferred to University Hospital for further evaluation and drainage of peritonsillar abscess.


---

